# 尝试使用 fasttext 进行文本分类

In [2]:
import jieba
import pandas as pd
import random

df_pos = pd.read_csv('./data/pos.csv', encoding='utf-8', sep='\t')
df_pos = df_pos.dropna()

df_neg = pd.read_csv('./data/neg.csv', encoding='utf-8', sep='\t')
df_neg = df_neg.dropna()


pos = df_pos.content.values.tolist()[10:25010]
neg = df_neg.content.values.tolist()[10:25010]

In [3]:
stopwords = pd.read_csv('./rowdata/stopwords.txt', index_col=False, quoting=3, 
                        names=['stopword'], sep='\t', encoding='utf-8')
stopwords = stopwords['stopword'].values

def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x: len(x)>1, segs)
            segs = filter(lambda x: x not in stopwords, segs)
            sentences.append('__label__' + str(category) + ' , ' + ' '.join(segs))
        except:
            print('wrong in here' + line)
            continue

In [4]:
cate_dic = {'pos': 1, 'neg': 2}

sentences = []

preprocess_text(pos, sentences, cate_dic['pos'])
preprocess_text(neg, sentences, cate_dic['neg'])


random.shuffle(sentences) # 打乱顺序， 类似洗牌

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/tb/k5xqd901235gk634ymvtk5140000gn/T/jieba.cache
Loading model cost 0.885 seconds.
Prefix dict has been built successfully.


In [7]:
len(sentences)*0.2  # 测试集长度， 四一拆分训练集与测试集

10000.0

In [8]:
print('将经过处理的数据写入') # split train data and test data

out = open('./data/fasttext_test_data_pos_neg.txt', 'w')
for sentence in sentences[:int(len(sentences)*0.2)]:
    out.write(sentence + '\n')

out = open('./data/fasttext_train_data_pos_neg.txt', 'w')
for sentence in sentences[int(len(sentences)*0.2 + 1):]:
    out.write(sentence + '\n')
    
print('done!')

将经过处理的数据写入
done!


In [9]:
import fasttext  
# fasttext 2. 与 3. 版本不同 ，此处为3.  
#https://github.com/facebookresearch/fastText/tree/master/python#train_supervised-parameters

classifier = fasttext.train_supervised('./data/fasttext_train_data_pos_neg.txt', label = '__label__')

In [10]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*classifier.test('./data/fasttext_train_data_pos_neg.txt'))  # 在训练样本进行测试  最初方法
print_results(*classifier.test('./data/fasttext_test_data_pos_neg.txt'))  # 在test data 进行测试

N	39955
P@1	0.944
R@1	0.944
N	10000
P@1	0.918
R@1	0.918


In [11]:
def comPreProcess(text):
    sentence = []
    segs = jieba.lcut(text)
    segs = filter(lambda x: len(x)>1, segs)
    segs = filter(lambda x: x not in stopwords, segs)
    sentence.append(' '.join(segs))
    return sentence[0]

comPreProcess("你是个大笨蛋，我讨厌你，你可太过分了啊")

'笨蛋 讨厌 太过分'

In [12]:
text = '你是个大笨蛋，我讨厌你，你可太过分了啊'
classifier.predict(comPreProcess(text), k = 2)

(('__label__2', '__label__1'), array([0.96607131, 0.03394863]))

In [13]:
df_pl_content = pd.read_csv('./data/BiliComment_contain20000.csv', sep='\t')

df_pl_content = df_pl_content.content.values.tolist()

for i in df_pl_content:
    print(i,classifier.predict(comPreProcess(i)))  # 除了分词还可以过滤下 停用词，写个函数啥的

也是上过央视哈哈哈哈哈哈 (('__label__1',), array([0.99821913]))
恶意抢注 (('__label__2',), array([0.87583774]))
加油 (('__label__1',), array([0.96855032]))
老爹说过要用魔法对抗魔法        出自《成龙历险记》。 (('__label__2',), array([0.62843812]))
支持维权 (('__label__1',), array([0.73434883]))
身份证不见了还可以去改名吗？急，打算明天去公安局改的，结果身份证不见了，真的想哭，已经成年了，只带户口本可以去改吗？ (('__label__2',), array([0.86561525]))
加油 (('__label__1',), array([0.96855032]))
真他妈的可悲 (('__label__2',), array([0.81926042]))
2507.4w (('__label__2',), array([0.95379311]))
一杯敬死神，一杯敬汉卿。 (('__label__2',), array([0.95379311]))
加油 (('__label__1',), array([0.96855032]))
加油 (('__label__1',), array([0.96855032]))
皮皮虾和b站都关注了 (('__label__1',), array([0.9913255]))
加油⛽️ (('__label__1',), array([0.96855032]))
冲冲冲 (('__label__2',), array([0.84212017]))
这事本身就很荒唐，人家名字也可以注册？呵呵笑死我了，这个部门在干嘛 (('__label__2',), array([0.95216334]))
真的没想到敬汉卿是真名，父母太有才了，取的好好听 (('__label__1',), array([0.92529881]))
第一次三连[OK] (('__label__1',), array([0.84938747]))
太难了up主 (('__label__2',), array([0.97364247]))
一直喜欢敬汉卿大大的视

加油！ (('__label__1',), array([0.96855032]))
捐去哪 (('__label__2',), array([0.95379311]))
那么问题来了，半年了敬汉卿官司结果出来了吗，百度查不到[无语] (('__label__2',), array([0.97618687]))
[老鼠] (('__label__2',), array([0.83469951]))
你可以，支持up主 (('__label__1',), array([0.98099518]))
加油(ง •̀_•́)ง，支持 (('__label__1',), array([0.99733901]))
哈哈 (('__label__2',), array([0.95379311]))
奥力给 (('__label__2',), array([0.95379311]))
加油 (('__label__1',), array([0.96855032]))
冲！ (('__label__2',), array([0.95379311]))
[打call][打call][打call][打call][打call] (('__label__1',), array([0.73283869]))
up加油↖(^ω^)↗ (('__label__1',), array([0.96855032]))
我想到了scp被人在俄罗斯抢注的事。 (('__label__2',), array([0.99320334]))
b站历史硬币数量第一 (('__label__2',), array([0.79954648]))
[热词系列_三连][热词系列_三连][热词系列_三连] (('__label__1',), array([0.9995808]))
加油 (('__label__1',), array([0.96855032]))
我在 (('__label__2',), array([0.95379311]))
这个视频给好多大Up主带火了，作为一名b站萌新我感到很温暖，真的 (('__label__1',), array([0.50298059]))
加油!!!! (('__label__1',), array([0.96855032]))
来一条评论，帮敬汉卿 (('__label__2

冲冲冲 (('__label__2',), array([0.84212017]))
这个播放量，哇贼恐惧 (('__label__2',), array([0.93858469]))
每周必看哈哈哈 (('__label__1',), array([0.97450817]))
？评论呢？ (('__label__2',), array([0.60836554]))
分享 了 催更催更 (('__label__2',), array([0.5636819]))
我宅b站支持你 (('__label__1',), array([0.98099518]))
加油 (('__label__1',), array([0.96855032]))
牛逼 (('__label__2',), array([0.95379311]))
加油加油 (('__label__1',), array([0.99624592]))
冲 (('__label__2',), array([0.95379311]))
你很出名！ (('__label__2',), array([0.95379311]))
我一直在看你，但是我发现我并没有关注你。尴尬，赶紧关注。[滑稽] (('__label__1',), array([0.91435933]))
敬汉卿你好，我也经常看你的节目，我们也是做商标这块的。刚才查询了一下，目前这个41还不是最要命的，你的35类也被抢注了，现在正在公示，如果35类被抢注，你就不能做广告，彻底凉凉了，希望你能引起共重视，现在赶紧去。35类的商标进行复议 复制自热心网友35类已经在2019.7.13号开始初审公示了，如果在三个月之内，也就是10月13号之前没有人复议的话就通过了，你35类商标如果丢了的话，你彻底就告别这个名字了（复制粘贴评论区） (('__label__2',), array([0.6438753]))
噢力给 (('__label__2',), array([0.95379311]))
加油，奥利给！ (('__label__1',), array([0.96855032]))
up主加油 虽然你普通话不好 (('__label__2',), array([0.85646349]))
666666666 (('__label__2',), array([0.9

加油 (('__label__1',), array([0.96855032]))
加油！ (('__label__1',), array([0.96855032]))
(ง •̀_•́)ง (('__label__2',), array([0.95379311]))
加油！干爆邪恶！ (('__label__1',), array([0.96855032]))
竟然是真名！！！ (('__label__2',), array([0.95379311]))
咋俩生日一样[微笑][微笑] (('__label__2',), array([0.86419016]))
这些人怎么这么恶心 (('__label__2',), array([1.00000107]))
如果up主B站号换名了咱就改名叫敬汉卿 (('__label__2',), array([0.95379311]))
加油_x1f4aa_ (('__label__1',), array([0.96855032]))
加油 (('__label__1',), array([0.96855032]))
[偷笑]你永远不知道会有什么名字被注册 (('__label__2',), array([0.923549]))
助你一币之力 (('__label__2',), array([0.81542051]))
加油 (('__label__1',), array([0.96855032]))
加油啊，会支持你的 (('__label__1',), array([0.99733901]))
你币有了 (('__label__2',), array([0.95379311]))
好的 (('__label__2',), array([0.95379311]))
让我想到了那个要你证明你麻麻是你麻麻那个话题[doge] (('__label__2',), array([0.53536063]))
。。 (('__label__2',), array([0.95379311]))
收个藏 (('__label__2',), array([0.81000203]))
加油 (('__label__1',), array([0.96855032]))
考古的时候看到标题我吓到了ʘᴗʘ敬汉卿太难了 (('__label__2',),

... (('__label__2',), array([0.95379311]))
支持 (('__label__1',), array([0.98099518]))
！！！！好 (('__label__2',), array([0.95379311]))
2349.4万 (('__label__2',), array([0.95379311]))
。。。。。。。。？ (('__label__2',), array([0.95379311]))
加油 (('__label__1',), array([0.96855032]))
支持 (('__label__1',), array([0.98099518]))
出名出名，绝对出名 (('__label__2',), array([0.95379311]))
卧槽 (('__label__2',), array([0.93599516]))
加油 (('__label__1',), array([0.96855032]))
他知道很出名 (('__label__2',), array([0.95379311]))
加油 (('__label__1',), array([0.96855032]))
来来来 (('__label__2',), array([0.62196231]))
2333.3万了 (('__label__2',), array([0.95379311]))
话不多说，三连走起 (('__label__2',), array([0.95379311]))
支持维权 (('__label__1',), array([0.73434883]))
哈哈哈 (('__label__1',), array([0.9961555]))
加油！ (('__label__1',), array([0.96855032]))
[2233娘_大笑] (('__label__2',), array([0.95379311]))
2333万[抠鼻] (('__label__2',), array([0.80685061]))
加油 (('__label__1',), array([0.96855032]))
可怜的主播大大。 (('__label__2',), array([0.8892175]))
整个b站有4千万人3百万

...

共青团中央今天看到了_x1f601_ (('__label__2',), array([0.95379311]))
加油！！！ (('__label__1',), array([0.96855032]))
支持！ (('__label__1',), array([0.98099518]))
来了 (('__label__2',), array([0.95379311]))
公道自在人心，支持合法维权。[支持] (('__label__1',), array([0.90004587]))
加油加油(ง •̀_•́)ง (('__label__1',), array([0.99624592]))
你真的好出名 (('__label__2',), array([0.92720616]))
打气 (('__label__2',), array([0.95379311]))
加油 (('__label__1',), array([0.96855032]))
加油！ (('__label__1',), array([0.96855032]))
加油！ (('__label__1',), array([0.96855032]))
支持维权 (('__label__1',), array([0.73434883]))
加油加油，无论怎样都会继续关注的[点赞] (('__label__1',), array([0.99983531]))
大家好，我是敬汉敬[大笑] (('__label__2',), array([0.95379311]))
支持你 (('__label__1',), array([0.98099518]))
加油_x1f4aa_ (('__label__1',), array([0.96855032]))
这些坏人 (('__label__2',), array([0.81519437]))
加油，别输啊 (('__label__1',), array([0.96855032]))
我透！敬汉卿只比我大三岁！ (('__label__2',), array([0.95379311]))
潜水党顶一下 (('__label__2',), array([0.7742911]))
天眼查也看到几个敬汉卿商标信息 (('__label__2',), array([0.968

法律真的好，维权保护，但是也有不好的，比如遇到这种人没办法一巴掌拍死。 (('__label__2',), array([0.99763769]))
支持维权！ (('__label__1',), array([0.73434883]))
这个名字真的很好听 (('__label__2',), array([0.96402878]))
稳住，我们能赢！ (('__label__2',), array([0.95379311]))
助战 (('__label__2',), array([0.95379311]))
这个事现在闹的这么火，应该是商家道歉的时候了吧 (('__label__2',), array([0.74856794]))
挺你！ (('__label__2',), array([0.95379311]))
加油加油 (('__label__1',), array([0.99624592]))
敬汉卿是个聪明人， (('__label__2',), array([0.95379311]))
加油 (('__label__1',), array([0.96855032]))
那我将来也要走主播这个行业，正好自己家也有一个公司，那我要不要先把自己注册了呀？求解答 (('__label__1',), array([0.98350394]))
哪个憨批诈骗公司？ (('__label__1',), array([0.96801108]))
加油！ (('__label__1',), array([0.96855032]))
前有香港爱国民众冒雨撑警察 今有B站网友撑敬汉卿 (('__label__1',), array([0.59753668]))
加油！ (('__label__1',), array([0.96855032]))
死神之子怎么可以被讹！！！！！！！！！！！！！ (('__label__2',), array([0.95379311]))
加油 (('__label__1',), array([0.96855032]))
顶起来 (('__label__2',), array([0.95379311]))
最近敬汉卿不更新了，在这里再刷几遍吧 (('__label__2',), array([0.58167213]))
支持up主 (('__l

我家在芜湖，离那家公司只有两千米的距离，有需要喊我一声 (('__label__1',), array([0.55723882]))
加油！加油！ (('__label__1',), array([0.99624592]))
很可怜一up主，但是真的很想笑。。。。加油，支持维权！ (('__label__1',), array([0.75158465]))
加油！ (('__label__1',), array([0.96855032]))
加油 (('__label__1',), array([0.96855032]))
镇站之宝,收藏很多[滑稽][滑稽] (('__label__1',), array([0.52964908]))
本人也是从事律师职业的，不要把商业简单的用法律套用，现在需要的是解决多个问题，一，通过法律途径拿回商标权，这需要大量的精力和时间，在此过程中，很可能损失惨重，这就是大部分恶意注册商标的寻租空间，硬刚的前提是不能伤敌一万自损八千。二，我的建议还是两条腿走路，支持在现在热度高的时候开新的号，新号内容一个就是把过去部分内容平移过去，这点上我想B站可以帮忙减少操作吧，另一个就是开个维权主题，趁热打铁，聚沙成塔，对类似问题做一个针对性的聚合，甚至可以做成一个公益组织，我想通过你的影响力和大家的支持是没有问题的。三，前两个不但可以把预期风险最小化，还可以正面加强维权力度，这是个好事。千万不要成为赢了案子，输了现实，商人的恶毒绝对超出法律的边界。 (('__label__1',), array([0.59262311]))
敬汉卿好像说过，他以前赌博输了很多。然后那段时间。拼命的想火，想赚钱。然后各种拼命，作死。从当初作死一直到现在....看了几年了。挺喜欢他的。现在日子也安定下来。愿汉卿维权成功（[doge][doge][doge][doge] (('__label__1',), array([0.67999429]))
评论 (('__label__2',), array([0.60836554]))
打气！！！！！ (('__label__2',), array([0.95379311]))
加油！ (('__label__1',), array([0.96855032]))
加油[支持] (('__label__1',), array([0

加油 (('__label__1',), array([0.96855032]))
我敬你敬汉卿是条汉子 (('__label__2',), array([0.95379311]))
odk (('__label__2',), array([0.95379311]))
支持合法维权(｀・ω・´) (('__label__1',), array([0.70394027]))
支持 (('__label__1',), array([0.98099518]))
顶，继续顶 (('__label__2',), array([0.95379311]))
加油！ (('__label__1',), array([0.96855032]))
支持合法维权！！！(=・ω・=) (('__label__1',), array([0.70394027]))
顶！[画风突变] (('__label__2',), array([0.82271379]))
加油 (('__label__1',), array([0.96855032]))
知名度杠杠的 (('__label__1',), array([0.99996114]))
这要拖更多久 (('__label__2',), array([0.97200596]))
加油 (('__label__1',), array([0.96855032]))
加油呀大大 (('__label__1',), array([0.96855032]))
加油 (('__label__1',), array([0.96855032]))
加油啊 (('__label__1',), array([0.96855032]))
居然涨了70w粉丝 (('__label__1',), array([0.71261793]))
惊了 (('__label__2',), array([0.95379311]))
加油汉卿！支持你！ (('__label__1',), array([0.99733901]))
加油！ (('__label__1',), array([0.96855032]))
加油(ง •̀_•́)ง (('__label__1',), array([0.96855032]))
这就是你拖跟视频的理由？[无语][无语][无语][无语][无语][无语][

只要不改名，你就是合法的呀，gang就完事了 (('__label__2',), array([0.8018989]))
加油[支持][支持] (('__label__1',), array([0.99924159]))
如何注册商标av62340999 (('__label__2',), array([0.95379311]))
。。 (('__label__2',), array([0.95379311]))
UP加油！！！！ (('__label__1',), array([0.96855032]))
我还是第一次见到这排面！！ (('__label__2',), array([0.77844536]))
现在是什么情况？ (('__label__2',), array([0.99748051]))
我取关你了 (('__label__2',), array([0.95379311]))
支持up主～ (('__label__1',), array([0.98099518]))
评论评论 (('__label__1',), array([0.60392964]))
唔，这类事情真的很多，而且还有一种是使用的字体....现在网络上很多字体都不是免费的，有的人用来做视频出名之后，那些所谓商家就找上门来了，都是一种套路 (('__label__2',), array([0.97242391]))
给你打打气 (('__label__2',), array([0.95379311]))
加油打气 (('__label__2',), array([0.95379311]))
知名度杠杠的 (('__label__1',), array([0.99996114]))
居然涨了70w粉丝 (('__label__1',), array([0.71261793]))
哈哈哈哈哈哈哈笑死我了，万万没想到是真名，父母给的 (('__label__1',), array([0.99205697]))
热评我划都划不完 (('__label__2',), array([0.81046408]))
不要在弹幕刷播放量 (('__label__2',), array([0.9579078]))
加油up主 (('__label__1',), array([0.96855032]))
支持